In [338]:
import pandas as pd
import numpy as np
import re
import json
from collections import Counter
from heapq import nlargest
import liwc
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer

import spacy
nlp = spacy.load("en_core_web_sm")

### Tag news article with POS & LIWC

In [27]:
news2 = pd.read_csv('data/kaggle_train.csv')
news2 = news2[~news2['text'].isnull()]
news2 = news2[[ 'title', 'author', 'text', 'label']]

news2['pos'] = news2['text'].apply(lambda x: collections.Counter([ w.pos_ for w in nlp(x)]))


def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)
def parseLIWC(x):
    gettysburg_tokens = tokenize(x)
    gettysburg_counts = Counter(category for token in gettysburg_tokens for category in parse(token))
    return gettysburg_counts


parse, category_names = liwc.load_token_parser('data/queryDictionary.dic')

news2['liwc_ct'] = news2['text'].apply(parseLIWC)


json_struct = json.loads(news2.to_json(orient="records"))    
df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

df_flat= df_flat.fillna(0)
df_flat['word_count'] = df_flat.loc[:, df_flat.columns.str.startswith('pos.')].sum(axis=1)

In [485]:
df_flat3.to_csv('data/flaten_file2.csv')

In [60]:
df_flat1 = pd.read_csv('data/flaten_file.csv')

In [57]:

df_flat[list(df_flat.columns[4:-1])] = (df_flat.T.iloc[4:-1] / df_flat.T.iloc[95]).T

In [364]:
pos = df_flat.loc[:, (df_flat.columns.str.startswith('pos.')|df_flat.columns.str.startswith('label'))]

In [365]:
pos_grouped = pos.groupby('label').sum().T

In [366]:
# 0:10387
# 1:10374
pos_grouped[0] = pos_grouped[0]/10387
pos_grouped[1] = pos_grouped[1]/10374

In [368]:
pos_grouped['delta'] =pos_grouped[1] -pos_grouped[0]#.T
pos_grouped = pos_grouped.sort_values(by='delta',ascending=False)

In [346]:
fal = pos_grouped[[0]]
fal['label']=['False']*len(pos_grouped)
fal['pos']=pos_grouped.index
fal = fal.rename(columns={0: 'value'})
tru = pos_grouped[[1]]
tru['label']=['True']*len(pos_grouped)
tru['pos']=pos_grouped.index
tru = tru.rename(columns={1: 'value'})

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [347]:
fal_tru = pd.concat([fal, tru],axis=0)

In [348]:
import altair as alt

alt.Chart(fal_tru).mark_bar().encode(
    x='label:O',
    y='value:Q',
    color='label:N',
    column='pos:N'
)

alt.Chart(...)

In [317]:
liwc_df = df_flat.loc[:, (df_flat.columns.str.startswith('liwc')|df_flat.columns.str.startswith('label'))]

liwc_grouped = liwc_df.groupby('label').sum().T
# 0:10387
# 1:10374
liwc_grouped[0] = liwc_grouped[0]/10387
liwc_grouped[1] = liwc_grouped[1]/10374

liwc_fal = liwc_grouped[[0]]
liwc_fal['label']=['False']*len(liwc_grouped)
liwc_fal['pos']=liwc_grouped.index
liwc_fal = liwc_fal.rename(columns={0: 'value'})
liwc_tru = liwc_grouped[[1]]
liwc_tru['label']=['True']*len(liwc_grouped)
liwc_tru['pos']=liwc_grouped.index
liwc_tru = liwc_tru.rename(columns={1: 'value'})

liwc_fal_tru = pd.concat([liwc_fal, liwc_tru],axis=0)


alt.Chart(liwc_fal_tru).mark_bar().encode(
    x='label:O',
    y='value:Q',
    color='label:N',
    column='pos:N'
)

alt.Chart(...)

In [350]:
liwc_grouped['delta'] = liwc_grouped[1]-liwc_grouped[0]
liwc_grouped = liwc_grouped.sort_values(by='delta',ascending=False)

In [358]:
liwc_df = df_flat.loc[:, (df_flat.columns.str.startswith('liwc')|df_flat.columns.str.startswith('label'))]
liwc_grouped = liwc_df.groupby('label').sum().T
# 0:10387
# 1:10374
liwc_grouped[0] = liwc_grouped[0]/10387
liwc_grouped[1] = liwc_grouped[1]/10374
liwc_grouped_chart_d = liwc_grouped.T[['liwc_ct.Certain', 'liwc_ct.Informal', 'liwc_ct.Cause', 'liwc_ct.Ipron','liwc_ct.Adj', 'liwc_ct.Prep', 'liwc_ct.Function']]
liwc_grouped_chart_d
            

,liwc_ct.Certain,liwc_ct.Informal,liwc_ct.Cause,liwc_ct.Ipron,liwc_ct.Adj,liwc_ct.Prep,liwc_ct.Function
label,,,,,,,
0,0.006602,0.001549,0.011203,0.031519,0.031442,0.119313,0.342533
1,0.008709,0.003032,0.012425,0.029884,0.029585,0.106406,0.316042


In [233]:
news = news2[['text','label']]

In [249]:
news_sampled = news.sample(frac=0.1, random_state=1)

In [305]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(pos, test_size=0.2)

In [408]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pos[list(pos.columns)[1:]], list(pos['label']), test_size=0.2, random_state=42)

names = [
         "Naive Bayes",
         "Linear SVM",
         "Logistic Regression",
         "Random Forest",
         "Multilayer Perceptron"
        ]

classifiers = [
    MultinomialNB(),
    LinearSVC(),
    LogisticRegression(),
    RandomForestClassifier(),
    MLPClassifier()
]

parameters = [
              {'clf__alpha': (10, 1, 1e-1, 1e-2, 1e-3)},
              {'clf__C': (np.logspace(-5, 1, 5))},
              {'clf__C': (np.logspace(-5, 1, 5))},
              {'clf__max_depth': (1, 2, 5, 10, 50)},
              {'clf__alpha': (10, 1, 1e-1,1e-2, 1e-3)}
             ]
result = []
for name, classifier, params in zip(names, classifiers, parameters):
    clf_pipe = Pipeline([
        #('vect', TfidfVectorizer(stop_words='english')),
        ('clf', classifier),
    ])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params,cv=5,n_jobs=3,verbose=2)
    clf = gs_clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    result.append([name,classifier,params,score,clf.best_params_, clf.best_score_, clf.cv_results_])
    

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  20 out of  25 | elapsed:    1.2s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    1.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Done  20 out of  25 | elapsed:    1.3s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    3.3s finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  20 out of  25 | elapsed:    1.1s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    1.4s finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:   23.5s finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:  1.0min finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(liwc_df[list(liwc_df.columns)[1:]], list(liwc_df['label']), test_size=0.2, random_state=42)

names = [
         "Naive Bayes",
         "Linear SVM",
         "Logistic Regression",
         "Random Forest",
         "Multilayer Perceptron"
        ]

classifiers = [
    MultinomialNB(),
    LinearSVC(),
    LogisticRegression(),
    RandomForestClassifier(),
    MLPClassifier()
]

parameters = [
              {'clf__alpha': (10, 1, 1e-1, 1e-2, 1e-3)},
              {'clf__C': (np.logspace(-5, 1, 5))},
              {'clf__C': (np.logspace(-5, 1, 5))},
              {'clf__max_depth': (1, 2, 5, 10, 50)},
              {'clf__alpha': (10, 1, 1e-1,1e-2, 1e-3)}
             ]
result_liwc = []
for name, classifier, params in zip(names, classifiers, parameters):
    clf_pipe = Pipeline([
        #('vect', TfidfVectorizer(stop_words='english')),
        ('clf', classifier),
    ])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params,cv=5,n_jobs=3,verbose=2)
    clf = gs_clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    result_liwc.append([name,classifier,params,score,clf.best_params_, clf.best_score_, clf.cv_results_])
    

NameError: name 'liwc_df' is not defined

In [421]:
parameters = [
              {'clf__max_depth': (1, 2, 5, 10, 50)}
             ]

clf_pipe = Pipeline([
        ('clf', RandomForestClassifier()),
    ])
gs_clf = GridSearchCV(clf_pipe, param_grid=parameters,cv=5,n_jobs=3,verbose=2)
clf = gs_clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)



Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:   25.0s finished


In [423]:
model = clf.best_estimator_

In [424]:
import pickle
pickle_out = open("classifier.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [445]:
pos_result = pd.DataFrame([[mod[0]]+list(mod[6].get('mean_test_score')) for mod in result], columns=['model','p1','p2','p3','p4','p5'])

In [479]:
pos_result_flatten.to_csv('data/pos_result_flatten.csv')
liwc_result_flatten.to_csv('data/liwc_result_flatten.csv')

In [446]:
liwc_result = pd.DataFrame([[mod[0]]+list(mod[6].get('mean_test_score')) for mod in result_liwc], columns=['model','p1','p2','p3','p4','p5'])

In [478]:
pos_result_flatten = pd.DataFrame()
for i in range(1,6):
    p1 = pos_result[['model','p'+str(i)]]
    p1['parameter']=['p'+str(i)]*len(pos_result)
    p1 = p1.rename(columns={'p'+str(i): 'acuracy'})
    if i == 1:
        pos_result_flatten = p1
    else:
        pos_result_flatten = pd.concat([pos_result_flatten,p1], axis=0)
pos_result_flatten

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,model,acuracy,parameter
0,Naive Bayes,0.588030,p1
1,Linear SVM,0.531732,p1
2,Logistic Regression,0.500241,p1
3,Random Forest,0.647820,p1
4,Multilayer Perceptron,0.499759,p1
0,Naive Bayes,0.599531,p2
1,Linear SVM,0.601337,p2
2,Logistic Regression,0.548832,p2
3,Random Forest,0.682502,p2
4,Multilayer Perceptron,0.591042,p2


In [474]:
liwc_result_flatten = pd.DataFrame()
for i in range(1,6):
    p1 = liwc_result[['model','p'+str(i)]]
    p1['parameter']=['p'+str(i)]*len(liwc_result)
    p1 = p1.rename(columns={'p'+str(i): 'acuracy'})
    if i == 1:
        liwc_result_flatten = p1
    else:
        liwc_result_flatten = pd.concat([liwc_result_flatten,p1], axis=0)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [477]:
liwc_result_flatten.to_csv('data/liwc_result.csv')# = liwc_result[['model','p'+str(i)]]

In [456]:
pd.concat([liwc_result[['model','p1']],liwc_result[['model','p2']]], axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,model,p1,p2
0,Naive Bayes,0.698520,NaN
1,Linear SVM,0.532093,NaN
2,Logistic Regression,0.500301,NaN
3,Random Forest,0.700927,NaN
4,Multilayer Perceptron,0.500000,NaN
0,Naive Bayes,NaN,0.704661
1,Linear SVM,NaN,0.594172
2,Logistic Regression,NaN,0.604890
3,Random Forest,NaN,0.736693
4,Multilayer Perceptron,NaN,0.697195


In [435]:
x = "adobochron 1 Comment SAN FRANCISCO, California ( The Adobo Chronicles, San Francisco Bureau) – Now it can be told — now that Hillary Clinton is all but assured of the Democratic presidential nomination. For months since the former secretary of state launched her presidential bid, her campaign logo has always symbolized America’s quest to move forward, with the arrow in the logo pointing to the right. Left-handed people, of course, dispute that symbolism because for them, moving forward should be pointing to the left. But there is another school of thought about Hillary’s logo. Many believe that the arrow pointing right actually symbolizes moving to the (conservative) right as opposed to the (liberal) left. The logo was a brilliant concept designed to help the Democrats win over Republican voters in the general elections — voters who will definitely choose a rightist over a leftist candidate. The GOP’s presumptive nominee, Donald Trump, is perceived by many as more leftist than rightist, which would give Clinton the edge among conservative voters. Who knew the placement or direction of an arrow could spell victory or defeat for an American presidential candidate? Rate this:"
x_parsed = parseLIWC(x)

liwc_ct = [x_parsed[name.replace("liwc_ct.","")] for name in X_train.columns]
liwc_sum = sum(liwc_ct)
liwc_ip = [ (xi*1.0)/liwc_sum for xi in liwc_ct]

AttributeError: 'list' object has no attribute 'sum'